# AMMI notes - ridge equality
> Solution to the optimization of the squarred error with accompanying code.

- toc:true
- badges: true
- comments: true
- author: Gbetondji Dovonon
- categories: [notes,ammi,blog]